In [1]:
import pandas as pd
import sklearn.model_selection as ms
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.preprocessing as pre
import sklearn.metrics as me
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import numpy as np

import pickle

In [2]:
RANDOM_STATE = 42

In [3]:
submission = pd.read_csv('../Data/files/stats_test.csv')
flights_engi = pd.read_csv('../Data/files/stats.csv')
flights_engi.head()

In [ ]:
submission.head()

In [ ]:
submission = submission.drop('branded_code_share', axis=1)

flights_engi = flights_engi.drop(columns=['branded_code_share'])
cols_to_test = flights_engi.columns

# preview
flights_engi.columns

In [ ]:
X = flights_engi.drop('arr_delay', axis=1)
y = flights_engi['arr_delay']

### Processing the data

In [ ]:
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, random_state=RANDOM_STATE, test_size=0.3)

In [ ]:
pipe = Pipeline(steps=[
    # ('poly', pre.PolynomialFeatures()),
    ('std_scaler', pre.StandardScaler()),
    ('pca', PCA(random_state=RANDOM_STATE)),
    ('gbr', GradientBoostingRegressor(
        random_state=RANDOM_STATE),
                                 ),
    ])

In [ ]:
hyperparameters = {
    'gbr__learning_rate': np.arange(0.1, 1, 0.3),
    'gbr__n_estimators': np.arange(100, 1000, 400)
}

In [ ]:
grid_search = ms.GridSearchCV(pipe, hyperparameters, verbose=1, n_jobs=4)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
model = grid_search.best_estimator_
model

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
me.mean_squared_error(y_test, y_pred)

In [ ]:
me.r2_score(y_test, y_pred)

In [ ]:
# Save the model
# pickle.dump(model, open('first_submission_gbr.sav', 'wb'))

There is a lot more to iterate! See the repo readme for future directions.

### Get submission predictions

In [ ]:
predicted_delay = model.predict(submission[cols_to_test])
submission['predicted_delay'] = predicted_delay

In [ ]:
cols_to_submit = ['fl_date', 'mkt_carrier', 'mkt_carrier_fl_num', 'origin', 'dest', 'predicted_delay']
submission.columns

In [ ]:
# Save the predictions for submission

# submission[cols_to_submit].to_csv('final_submission.csv')